In [1]:
# Scientific libraries
import numpy as np
import scipy


# Graphic libraries

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("presentation")

# Creating alias for magic commands

# LPPview Classes
from LPPview import *
import pandas as pd

from tqdm import tqdm_notebook as tqdm

In [2]:
from datetime import datetime
import matplotlib.dates as mdates
from  scipy.ndimage import gaussian_filter1d as smooth 

In [3]:
df = pd.read_csv("./2ON01FR-20181105_-_station_sol_X.txt")
df["date"] = pd.to_datetime(df.time.values - datetime.timestamp(pd.to_datetime('2000-01-01 00:00:0'))
                                + datetime.timestamp(pd.to_datetime('2017-05-17 01:45:43')), unit='s') 

df.head()

,time,vGS4,iGS4,tGS4,vGS1,tGS1,iGS1,tBat,vBat,vGS2,...,RSSI,iTX,tPA,i1200,i3v3,v3v3,i5v,v5v,tSU,date
0,946693603,35,0,-22.1,35,-18.9,0,-1.3,8069,35,...,1224,0,14.7,0,0,0,0,0,0.0,2017-05-17 03:12:26
1,946693633,35,0,-22.1,35,-18.9,0,-1.3,8069,35,...,1264,0,13.1,0,0,0,0,0,0.0,2017-05-17 03:12:56
2,946693694,35,0,-23.7,35,-20.5,0,-1.3,8069,35,...,1200,0,13.1,0,0,0,0,0,0.0,2017-05-17 03:13:57
3,946693724,35,0,-23.7,35,-20.5,0,-1.3,8069,35,...,1200,0,13.1,0,0,0,0,0,0.0,2017-05-17 03:14:27
4,946693739,35,0,-23.7,35,-20.5,0,-1.3,8069,35,...,1328,0,13.1,0,0,0,0,0,0.0,2017-05-17 03:14:42


In [4]:
df["hour"] = pd.DatetimeIndex("2019-01-01 "+df.date.dt.hour.astype(str)+':'+
                              df.date.dt.minute.astype(str)+":"+
                              df.date.dt.second.astype(str),)

df["hour"] = pd.DatetimeIndex("2019-01-01 "+df.date.dt.hour.astype(str)+':'+
                              df.date.dt.minute.astype(str)+":"+
                              df.date.dt.second.astype(str),)

In [5]:
df.keys()

Index(['time', 'vGS4', 'iGS4', 'tGS4', 'vGS1', 'tGS1', 'iGS1', 'tBat', 'vBat',
       'vGS2', 'tGS2', 'iGS2', 'vGS3', 'tGS3', 'iGS3', 'iADCS', 'tODB', 'iRX',
       'RSSI', 'iTX ', 'tPA', 'i1200', 'i3v3', 'v3v3', 'i5v', 'v5v', 'tSU',
       'date', 'hour'],
      dtype='object')

# Temperature

We had some noise to the data for smoother visualisation

In [6]:
df['mean_tBat'] = smooth(df['tBat'], 50)
df['error_mean_tBat'] = df['tBat'] - df['mean_tBat'] 

df['tBat_noise'] = df['tBat'] + np.random.randn(len(df['tBat']))*1

In [14]:
from bokeh.plotting import figure, output_file, show
from bokeh.embed import json_item
from bokeh.models import BoxSelectTool
from bokeh.models import DatetimeTickFormatter
from bokeh.embed import autoload_static
from bokeh.resources import CDN

In [77]:
# output to static HTML file
output_file("tbat_all.html")

# create a new plot with a title and axis labels
p = figure(title="Temprature of the Battery along the mission",
           x_axis_label="Date",
           y_axis_label="Temperature [°C] ",
           tools="pan,wheel_zoom,box_zoom,reset",
           width=5,
           height=3)
p.add_tools(BoxSelectTool())
p.sizing_mode = 'scale_width'

# add a line renderer with legend and line thickness
#p.circle(x=df.hour, y=df.tBat, legend_label="Temp.", line_width=2)
p.circle(x=df.date, y=df.tBat, line_width=2)

p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )
p.xaxis.major_label_orientation = np.pi/4
p.toolbar.logo = None
# show the results
show(p)

In [78]:
# save the figure
js_filename = "/js/tbat_all.js"
js, tag = autoload_static(p, CDN, js_filename)

print(tag)

jsname = "../../static"+js_filename

try :
    with open(jsname, "x") as f:
        f.write(js)
except FileExistsError:
    with open(jsname, "w") as f:
        f.write(js)



<script src="/js/tbat_all.js" id="59e286ae-f060-4590-a82e-5382d49d87f7"></script>


In [79]:
# output to static HTML file
output_file("tbat.html")

from bokeh.models import BoxSelectTool
from bokeh.models import DatetimeTickFormatter

# create a new plot with a title and axis labels
p = figure(title="Temprature of the Battery during the day",
           x_axis_label="Local time in the day",
           y_axis_label="Temperature [°C] ",
           tools="pan,wheel_zoom,box_zoom,reset",
           width=5,
           height=3)

p.add_tools(BoxSelectTool())
p.sizing_mode = 'scale_width'

# add a line renderer with legend and line thickness
p.circle(x=df.hour, y=df.tBat, line_width=2)

p.xaxis.formatter=DatetimeTickFormatter(
        hourmin = ['%H:%M'],
        hours = ['%H:%M'],
    )
p.toolbar.logo = None
# show the results
#show(p)

In [80]:
# save the figure
js_filename = "/js/tbat_hour.js"
js, tag = autoload_static(p, CDN, js_filename)

print(tag)

jsname = "../../static"+js_filename

try :
    with open(jsname, "x") as f:
        f.write(js)
except FileExistsError:
    with open(jsname, "w") as f:
        f.write(js)



<script src="/js/tbat_hour.js" id="45613fa5-5614-42fe-94bc-872577b9f737"></script>
